In [1]:
import mxnet as mx
import numpy as np
import os, time, shutil

from mxnet import gluon, image, init, nd
from mxnet import autograd as ag
from mxnet.gluon import nn
from mxnet.gluon.data.vision import transforms
from gluoncv.utils import makedirs
from gluoncv.model_zoo import get_model

# 分割一下数据集~

In [2]:
imglist = os.listdir("./classified-images/normal/") + os.listdir("./classified-images/terror/")

trainPlusValNum = (terrorNum + normalNum)*0.7 # 8117
testNum = (terrorNum + normalNum)*0.3 # 3479

normalNum = 1335
terrorNum = 10261

NameError: name 'terrorNum' is not defined

# set some parameters

In [2]:
classes = 2

epochs = 40
lr = 0.001
per_device_batch_size = 256
momentum = 0.9
wd = 0.0001

lr_factor = 0.75
lr_steps = [10, 20, 30, np.inf]

num_gpus = 1
num_workers = 8
ctx = [mx.gpu(i) for i in range(num_gpus)] if num_gpus > 0 else [mx.cpu()]
batch_size = per_device_batch_size * max(num_gpus, 1)

# data augmentation

In [3]:
jitter_param = 0.4
lighting_param = 0.1

transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomFlipLeftRight(),
    transforms.RandomColorJitter(brightness=jitter_param, contrast=jitter_param,
                                 saturation=jitter_param),
    transforms.RandomLighting(lighting_param),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# define dataloader

In [4]:
from mxnet.gluon.data.vision import ImageFolderDataset
import cv2

class BKData(ImageFolderDataset):
    def __init__(self, *arg1, **arg2):
        super(BKData,self).__init__(*arg1,**arg2)

    def __getitem__(self, idx):
        '''
        use cv2 backend
        '''
        img  = cv2.imread(self.items[idx][0])
        img  = nd.array( img[:,:,:3]).astype(np.uint8)
        label = self.items[idx][1]
        if self._transform is not None:
            return self._transform(img, label)
        return img, label

In [5]:
path = './images'
train_path = os.path.join(path, 'train')
val_path = os.path.join(path, 'val')
test_path = os.path.join(path, 'test')

train_data = gluon.data.DataLoader(
    BKData(train_path).transform_first(transform_train),
    batch_size=batch_size, shuffle=True, num_workers=num_workers)

val_data = gluon.data.DataLoader(
    BKData(val_path).transform_first(transform_test),
    batch_size=batch_size, shuffle=False, num_workers = num_workers)

test_data = gluon.data.DataLoader(
    BKData(test_path).transform_first(transform_test),
    batch_size=batch_size, shuffle=False, num_workers = num_workers)

# model and trainer

In [6]:
model_name = 'ResNet50_v2'
finetune_net = get_model(model_name, pretrained=True)
with finetune_net.name_scope():
    finetune_net.output = nn.Dense(classes)
finetune_net.output.initialize(init.Xavier(), ctx = ctx)
# finetune_net.cast('float16')
finetune_net.collect_params().reset_ctx(ctx)
finetune_net.hybridize()

trainer = gluon.Trainer(finetune_net.collect_params(), 'sgd', {
                        'learning_rate': lr, 'momentum': momentum, 'wd': wd})
metric = mx.metric.Accuracy()
L = gluon.loss.SoftmaxCrossEntropyLoss()


trainer = gluon.Trainer(finetune_net.collect_params(), 'sgd', {
                        'learning_rate': lr, 'momentum': momentum, 'wd': wd,
                        'multi_precision': True})



metric = mx.metric.Accuracy()
L = gluon.loss.SoftmaxCrossEntropyLoss()

In [7]:
def test(net, val_data, ctx):
    metric = mx.metric.Accuracy()
    for i, batch in enumerate(val_data):
        data = gluon.utils.split_and_load(batch[0].astype("float16", copy=False), ctx_list=ctx, batch_axis=0, even_split=False)
        label = gluon.utils.split_and_load(batch[1].astype("float16", copy=False), ctx_list=ctx, batch_axis=0, even_split=False)
        
        outputs = [net(X) for X in data]
        metric.update(label, outputs)

    return metric.get()

In [8]:
lr_counter = 0
num_batch = len(train_data)

time_duration = []
import time

for epoch in range(epochs):
    if epoch == lr_steps[lr_counter]:
        trainer.set_learning_rate(trainer.learning_rate*lr_factor)
        lr_counter += 1

    tic = time.time()
    train_loss = 0
    metric.reset()

    for i, batch in enumerate(train_data):
        data = gluon.utils.split_and_load(batch[0].astype("float16"), ctx_list=ctx, batch_axis=0, even_split=False)
        label = gluon.utils.split_and_load(batch[1].astype("float16"), ctx_list=ctx, batch_axis=0, even_split=False)
        with ag.record():
            t0 = time.time()
            outputs = [finetune_net(X) for X in data]
            time_duration.append(time.time() - t0)
            loss = [L(yhat, y) for yhat, y in zip(outputs, label)]
        for l in loss:
            l.backward()

        trainer.step(batch_size)
        train_loss += sum([l.mean().asscalar() for l in loss]) / len(loss)

        metric.update(label, outputs)

    _, train_acc = metric.get()
    train_loss /= num_batch

    _, val_acc = test(finetune_net, val_data, ctx)

    print('[Epoch %d] Train-acc: %.3f, loss: %.3f | Val-acc: %.3f | time: %.1f' %
             (epoch, train_acc, train_loss, val_acc, time.time() - tic))

print("Training Duration is {}".format(sum(time_duration)))
_, test_acc = test(finetune_net, test_data, ctx)
print('[Finished] Test-acc: %.3f' % (test_acc))

[Epoch 0] Train-acc: 0.864, loss: 0.415 | Val-acc: 0.885 | time: 71.8
[Epoch 1] Train-acc: 0.886, loss: 0.339 | Val-acc: 0.887 | time: 63.9
[Epoch 2] Train-acc: 0.888, loss: 0.306 | Val-acc: 0.891 | time: 64.8
[Epoch 3] Train-acc: 0.894, loss: 0.279 | Val-acc: 0.892 | time: 63.5
[Epoch 4] Train-acc: 0.897, loss: 0.270 | Val-acc: 0.894 | time: 63.4
[Epoch 5] Train-acc: 0.901, loss: 0.250 | Val-acc: 0.902 | time: 63.5
[Epoch 6] Train-acc: 0.906, loss: 0.245 | Val-acc: 0.901 | time: 63.3
[Epoch 7] Train-acc: 0.908, loss: 0.235 | Val-acc: 0.902 | time: 61.4
[Epoch 8] Train-acc: 0.911, loss: 0.228 | Val-acc: 0.905 | time: 63.5
[Epoch 9] Train-acc: 0.917, loss: 0.217 | Val-acc: 0.907 | time: 62.4
[Epoch 10] Train-acc: 0.921, loss: 0.210 | Val-acc: 0.906 | time: 63.7
[Epoch 11] Train-acc: 0.924, loss: 0.200 | Val-acc: 0.906 | time: 62.9
[Epoch 12] Train-acc: 0.921, loss: 0.204 | Val-acc: 0.909 | time: 63.5
[Epoch 13] Train-acc: 0.926, loss: 0.194 | Val-acc: 0.909 | time: 64.4
[Epoch 14] Train

In [10]:
finetune_net.save_parameters("./fp16-model.h5")